# Exercise 3
This exercise aims to examine a set of video files using ffprobe, and convert them into the specified format using ffmpeg.
## Generating a report of problematic fields
The following code analyses the ffprobe output of each video file and lists the fields that do not match the specified format in a text file report.txt.

In [1]:
import subprocess
import re
from fractions import Fraction

video_files = ['Cosmos_War_of_the_Planets.mp4', 'Last_man_on_earth_1964.mov', 'The_Gun_and_the_Pulpit.avi', 'The_Hill_Gang_Rides_Again.mp4', 'Voyage_to_the_Planet_of_Prehistoric_Women.mp4']

# Open report.txt in write mode
with open('report.txt', 'w') as file:
    for video_file in video_files:
        # Run ffprobe command
        command = f'ffprobe {video_file} -hide_banner'
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT, text=True)

        stream_indexes = []
        start = 0

        # Get starting index of each stream
        while True:
            start = output.find('Stream', start)

            if start == -1:
                break

            stream_indexes.append(start)
            start += 1

        streams = {}

        # Save each stream in a dictionary
        for stream_index in stream_indexes:
            type = output[stream_index + output[stream_index :].find(': ') + 2 : stream_index + output[stream_index :].find(': ') + 2 + output[stream_index + output[stream_index :].find(': ') + 2 :].find(':')]
            streams[type] = stream_index

        # Get each field from the output
        video_codec = output[streams['Video'] + output[streams['Video'] :].find('Video') + 7 : streams['Video'] + output[streams['Video'] :].find(' (')]
        audio_codec = output[streams['Audio'] + output[streams['Audio'] :].find('Audio') + 7 : streams['Audio'] + output[streams['Audio'] :].find(' (')]
        frame_rate = output[output[: output.find('fps')].rfind(',') + 2 : output.find(' fps')]
        resolution = re.search(r'\d{3,4}x\d{3,4}', output).group(0)
        video_bit_rate = output[output[: streams['Video'] + output[streams['Video'] :].find('kb/s')].rfind(',') + 2 : streams['Video'] + output[streams['Video'] :].find(' kb/s')]
        audio_bit_rate = output[output[: streams['Audio'] + output[streams['Audio'] :].find('kb/s')].rfind(',') + 2 : streams['Audio'] + output[streams['Audio'] :].find(' kb/s')]
        audio_channels = output[streams['Audio'] + output[streams['Audio'] :].find('Hz') + 4 : streams['Audio'] + output[streams['Audio'] :].find('Hz') + 4 + output[streams['Audio'] + output[streams['Audio'] :].find('Hz') + 4 :].find(',')]

        aspect_ratio_index = output.find('DAR')

        if aspect_ratio_index != -1:
            aspect_ratio = output[aspect_ratio_index + 4 : aspect_ratio_index + output[aspect_ratio_index :].find(',')]

            if aspect_ratio[-1] == ']':
                aspect_ratio = aspect_ratio[:-1]

        else:
            width, height = resolution.split('x')
            aspect_ratio_fraction = Fraction(int(width), int(height))
            aspect_ratio = f'{aspect_ratio_fraction.numerator}:{aspect_ratio_fraction.denominator}'
        
        # If field does not match the specified format, write it in the report.txt file
        file.write(f'{video_file}\n')
        
        if video_file.split('.')[1] != 'mp4':
            file.write(f'Video format (container) is {video_file.split(".")[1]} instead of mp4\n')
            
        if video_codec != 'h264':
            file.write(f'Video codec is {video_codec} instead of h264\n')
            
        if audio_codec != 'aac':
            file.write(f'Audio codec is {audio_codec} instead of aac\n')
            
        if frame_rate != '25':
            file.write(f'Frame rate is {frame_rate} fps instead of 25 fps\n')
            
        if aspect_ratio != '16:9':
            file.write(f'Aspect ratio is {aspect_ratio} instead of 16:9\n')
            
        if resolution != '640x360':
            file.write(f'Resolution is {resolution} instead of 640x360\n')
            
        if int(video_bit_rate) < 2000 or int(video_bit_rate) > 5000:
            file.write(f'Video bit rate is {video_bit_rate} kb/s, should be between 2000-5000 kb/s\n')
            
        if int(audio_bit_rate) > 256:
            file.write(f'Audio bit rate is {audio_bit_rate} kb/s, should be a maximum of 256 kb/s\n')
            
        if audio_channels != 'stereo' and audio_channels != '2 channels':
            file.write(f'Audio channels is {audio_channels} instead of stereo\n')
            
        file.write('\n')

## Converting problematic fields using ffmpeg
After identifying the problematic fields, the next step is to convert them into the specified format.
### Cosmos_War_of_the_Planets.mp4
For this video file, I converted the frame rate, aspect ratio, resolution and audio bit rate. I also converted the video bit rate as it fell out of range after the other fields were converted.

In [2]:
!ffmpeg -hide_banner -y -i Cosmos_War_of_the_Planets.mp4 -r 25 -aspect 16:9 -s 640x360 -b:v 3000k -b:a 256k Cosmos_War_of_the_Planets_formatOK.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Cosmos_War_of_the_Planets.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T19:15:48.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 3315 kb/s
  Stream #0:0[0x1](eng): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(progressive), 628x354 [SAR 1:1 DAR 314:177], 2989 kb/s, 29.97 fps, 29.97 tbr, 30k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T19:15:48.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : AVC Coding
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 317 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T19:15:48.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 ->

#### Before

In [3]:
!ffprobe Cosmos_War_of_the_Planets.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Cosmos_War_of_the_Planets.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T19:15:48.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 3315 kb/s
  Stream #0:0[0x1](eng): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(progressive), 628x354 [SAR 1:1 DAR 314:177], 2989 kb/s, 29.97 fps, 29.97 tbr, 30k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T19:15:48.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : AVC Coding
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 317 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T19:15:48.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]


#### After

In [4]:
!ffprobe Cosmos_War_of_the_Planets_formatOK.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Cosmos_War_of_the_Planets_formatOK.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.16.100
  Duration: 00:00:20.08, start: 0.000000, bitrate: 3227 kb/s
  Stream #0:0[0x1](eng): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 640x360 [SAR 1:1 DAR 16:9], 2975 kb/s, 25 fps, 25 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.31.102 libx264
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 245 kb/s (default)
    Metadata:
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]


### Last_man_on_earth_1964.mov
For this video file, I converted the video format (container), video codec, audio codec, frame rate, video bit rate and audio bit rate.

In [5]:
!ffmpeg -hide_banner -y -i Last_man_on_earth_1964.mov -r 25 -b:v 2000k Last_man_on_earth_1964_formatOK.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Last_man_on_earth_1964.mov':
  Metadata:
    major_brand     : qt  
    minor_version   : 537199360
    compatible_brands: qt  
    creation_time   : 2021-08-02T18:38:19.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 11240 kb/s
  Stream #0:0[0x1](eng): Video: prores (LT) (apcs / 0x73637061), yuv422p10le(smpte170m/smpte170m/bt709, progressive), 640x360, 9285 kb/s, SAR 1:1 DAR 16:9, 23.98 fps, 23.98 tbr, 24k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T18:38:19.000000Z
      handler_name    : Apple Video Media Handler
      vendor_id       : appl
      encoder         : Apple ProRes 422 LT
      timecode        : 00:51:43:11
  Stream #0:1[0x2](eng): Audio: pcm_s16le (sowt / 0x74776F73), 48000 Hz, stereo, s16, 1536 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T18:38:19.000000Z
      handler_name    : Apple Sound Media Handler
      vendor_id       :     
      timecode        : 00:51:43:11
  Stream #0

#### Before

In [6]:
!ffprobe Last_man_on_earth_1964.mov -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Last_man_on_earth_1964.mov':
  Metadata:
    major_brand     : qt  
    minor_version   : 537199360
    compatible_brands: qt  
    creation_time   : 2021-08-02T18:38:19.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 11240 kb/s
  Stream #0:0[0x1](eng): Video: prores (LT) (apcs / 0x73637061), yuv422p10le(smpte170m/smpte170m/bt709, progressive), 640x360, 9285 kb/s, SAR 1:1 DAR 16:9, 23.98 fps, 23.98 tbr, 24k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T18:38:19.000000Z
      handler_name    : Apple Video Media Handler
      vendor_id       : appl
      encoder         : Apple ProRes 422 LT
      timecode        : 00:51:43:11
  Stream #0:1[0x2](eng): Audio: pcm_s16le (sowt / 0x74776F73), 48000 Hz, stereo, s16, 1536 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T18:38:19.000000Z
      handler_name    : Apple Sound Media Handler
      vendor_id       :     
      timecode        : 00:51:43:11
  Stream #0

#### After

In [7]:
!ffprobe Last_man_on_earth_1964_formatOK.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Last_man_on_earth_1964_formatOK.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.16.100
  Duration: 00:00:20.04, start: 0.000000, bitrate: 2220 kb/s
  Stream #0:0[0x1](eng): Video: h264 (High 4:2:2) (avc1 / 0x31637661), yuv422p10le(tv, smpte170m/smpte170m/bt709, progressive), 640x360 [SAR 1:1 DAR 16:9], 2085 kb/s, 25 fps, 25 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : Apple Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.31.102 libx264
      timecode        : 00:51:43:11
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : Apple Sound Media Handler
      vendor_id       : [0][0][0][0]
  Stream #0:2[0x3](eng): Data: none (tmcd / 0x64636D74), 0 kb/s
    Metadata:
      handler_name    : Apple Video Media Handler

### The_Gun_and_the_Pulpit.avi
For this video file, I converted the video format (container), video codec, audio codec, aspect ratio, resolution, video bit rate, audio bit rate and audio channels.

In [8]:
!ffmpeg -hide_banner -y -i The_Gun_and_the_Pulpit.avi -aspect 16:9 -s 640x360 -b:v 3000k The_Gun_and_the_Pulpit_formatOK.mp4

[aist#0:1/pcm_s16le @ 0x1457062e0] Guessed Channel Layout: stereo
Input #0, avi, from 'The_Gun_and_the_Pulpit.avi':
  Duration: 00:00:20.00, start: 0.000000, bitrate: 88809 kb/s
  Stream #0:0: Video: rawvideo (IYUV / 0x56555949), yuv420p, 720x404, 87438 kb/s, 25 fps, 25 tbr, 25 tbn
  Stream #0:1: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 2 channels, s16, 1536 kb/s
Stream mapping:
  Stream #0:0 -> #0:0 (rawvideo (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (pcm_s16le (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x145707810] using SAR=1/1
[libx264 @ 0x145707810] using cpu capabilities: ARMv8 NEON
[libx264 @ 0x145707810] profile High, level 3.0, 4:2:0, 8-bit
[libx264 @ 0x145707810] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzo

#### Before

In [9]:
!ffprobe The_Gun_and_the_Pulpit.avi -hide_banner

Input #0, avi, from 'The_Gun_and_the_Pulpit.avi':
  Duration: 00:00:20.00, start: 0.000000, bitrate: 88809 kb/s
  Stream #0:0: Video: rawvideo (IYUV / 0x56555949), yuv420p, 720x404, 87438 kb/s, 25 fps, 25 tbr, 25 tbn
  Stream #0:1: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 2 channels, s16, 1536 kb/s


#### After

In [10]:
!ffprobe The_Gun_and_the_Pulpit_formatOK.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'The_Gun_and_the_Pulpit_formatOK.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.16.100
  Duration: 00:00:20.00, start: 0.000000, bitrate: 3047 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 640x360 [SAR 1:1 DAR 16:9], 2911 kb/s, 25 fps, 25 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.31.102 libx264
  Stream #0:1[0x2](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]


### The_Hill_Gang_Rides_Again.mp4
For this video file, I converted the video bit rate.

In [11]:
!ffmpeg -hide_banner -y -i The_Hill_Gang_Rides_Again.mp4 -b:v 5000k The_Hill_Gang_Rides_Again_formatOK.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'The_Hill_Gang_Rides_Again.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T18:46:51.000000Z
  Duration: 00:00:20.00, start: 0.000000, bitrate: 7799 kb/s
  Stream #0:0[0x1](eng): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(progressive), 640x360 [SAR 1:1 DAR 16:9], 7537 kb/s, 25 fps, 25 tbr, 25k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T18:46:51.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : AVC Coding
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 253 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T18:46:51.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aa

#### Before

In [12]:
!ffprobe The_Hill_Gang_Rides_Again.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'The_Hill_Gang_Rides_Again.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T18:46:51.000000Z
  Duration: 00:00:20.00, start: 0.000000, bitrate: 7799 kb/s
  Stream #0:0[0x1](eng): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(progressive), 640x360 [SAR 1:1 DAR 16:9], 7537 kb/s, 25 fps, 25 tbr, 25k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T18:46:51.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : AVC Coding
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 253 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T18:46:51.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]


#### After

In [13]:
!ffprobe The_Hill_Gang_Rides_Again_formatOK.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'The_Hill_Gang_Rides_Again_formatOK.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.16.100
  Duration: 00:00:20.02, start: 0.000000, bitrate: 5120 kb/s
  Stream #0:0[0x1](eng): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 640x360 [SAR 1:1 DAR 16:9], 4987 kb/s, 25 fps, 25 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.31.102 libx264
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]


### Voyage_to_the_Planet_of_Prehistoric_Women.mp4
For this video file, I converted the video codec, audio codec, frame rate, video bit rate and audio bit rate.

In [14]:
!ffmpeg -hide_banner -y -i Voyage_to_the_Planet_of_Prehistoric_Women.mp4 -vcodec h264 -acodec aac -r 25 -b:v 5000k -b:a 256k Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Voyage_to_the_Planet_of_Prehistoric_Women.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T19:26:10.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 8365 kb/s
  Stream #0:0[0x1](eng): Video: hevc (Main) (hvc1 / 0x31637668), yuv420p(tv, bt709), 640x360 [SAR 1:1 DAR 16:9], 8038 kb/s, 29.97 fps, 29.97 tbr, 30k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T19:26:10.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : HEVC Coding
  Stream #0:1[0x2](eng): Audio: mp3 (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 320 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T19:26:10.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (hevc (native) -> h264 (libx264))
  Stream 

#### Before

In [15]:
!ffprobe Voyage_to_the_Planet_of_Prehistoric_Women.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Voyage_to_the_Planet_of_Prehistoric_Women.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: mp42mp41
    creation_time   : 2021-08-02T19:26:10.000000Z
  Duration: 00:00:20.02, start: 0.000000, bitrate: 8365 kb/s
  Stream #0:0[0x1](eng): Video: hevc (Main) (hvc1 / 0x31637668), yuv420p(tv, bt709), 640x360 [SAR 1:1 DAR 16:9], 8038 kb/s, 29.97 fps, 29.97 tbr, 30k tbn (default)
    Metadata:
      creation_time   : 2021-08-02T19:26:10.000000Z
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : HEVC Coding
  Stream #0:1[0x2](eng): Audio: mp3 (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 320 kb/s (default)
    Metadata:
      creation_time   : 2021-08-02T19:26:10.000000Z
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]


#### After

In [16]:
!ffprobe Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4 -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.16.100
  Duration: 00:00:20.08, start: 0.000000, bitrate: 5052 kb/s
  Stream #0:0[0x1](eng): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 640x360 [SAR 1:1 DAR 16:9], 4803 kb/s, 25 fps, 25 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : ?Mainconcept Video Media Handler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.31.102 libx264
  Stream #0:1[0x2](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 242 kb/s (default)
    Metadata:
      handler_name    : #Mainconcept MP4 Sound Media Handler
      vendor_id       : [0][0][0][0]
